In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
from woodwork.logical_types import Categorical

In [2]:
# load dataFrame
clients_df = pd.read_csv("data/clients.csv")
loans_df = pd.read_csv("data/loans.csv")
payments_df = pd.read_csv("data/payments.csv")# Create new entity-set
es = ft.EntitySet(id="clients")

es

Entityset: clients
  DataFrames:
  Relationships:
    No relationships

In [3]:
# Attribute 'entity_from_dataframe' is OLD version
# Attribute 'add_dataframe' is NEW version
# Add an entity to entity-set
es = es.add_dataframe(dataframe_name="clients",
                      dataframe=clients_df,
                      index="client_id",
                      time_index="joined")

es = es.add_dataframe(dataframe_name="loans",
                      dataframe=loans_df,
                      logical_types={"repaid": Categorical},
                      index="loan_id",
                      time_index="loan_start")

es = es.add_dataframe(dataframe_name="payments",
                      dataframe=payments_df,
                      logical_types={"missed": Categorical},
                      make_index=True,
                      index="payment_id",
                      time_index="payment_date")

es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [4]:
# Aggregation
# Group loans by client id and calculate total of loans
stats = loans_df.groupby('client_id')['loan_amount'].agg(['sum'])
stats.columns = ['total_loan_amount']

# Merge with the clients dataframe
stats = clients_df.merge(stats, left_on='client_id', right_index=True, how="left")
stats.head(10)



,client_id,joined,income,credit_score,total_loan_amount
42320,42320,2000-04-27,229481,563,105931
39384,39384,2000-06-18,191204,617,149444
26945,26945,2000-11-26,214516,806,106889
41472,41472,2001-11-06,152214,638,120173
46180,46180,2001-11-06,43851,562,154017
46109,46109,2002-04-16,172677,527,179032
32885,32885,2002-05-13,58955,642,148806
29841,29841,2002-08-17,38354,523,176634
38537,38537,2002-10-21,127183,643,152768
35214,35214,2003-08-08,95849,696,129124


In [5]:
# --------------------------------------------------
# Relationships Between Entities
# Between clients and previous loans
r_client_previous = ft.Relationship(es, 'clients', 'client_id', 'loans', 'client_id')

# Between previous loans and previous payments
r_payments = ft.Relationship(es, 'loans', 'loan_id', 'payments', 'loan_id')

# Add the relationship to the entity set
es = es.add_relationship(relationship=r_client_previous)
es = es.add_relationship(relationship=r_payments)

es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [6]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset=es,
                                 target_dataframe_name="clients",
                                 agg_primitives=["sum"],
                                 trans_primitives=['year', 'month'])
features

,income,credit_score,SUM(loans.loan_amount),SUM(loans.rate),SUM(payments.payment_amount),MONTH(joined),YEAR(joined),SUM(payments.loans.loan_amount),SUM(payments.loans.rate)
client_id,,,,,,,,,
42320,229481,563,105931.0,36.86,122578.0,4,2000,843367.0,302.84
39384,191204,617,149444.0,67.23,174270.0,6,2000,1161741.0,499.12
26945,214516,806,106889.0,42.83,124261.0,11,2000,771053.0,330.13
41472,152214,638,120173.0,63.70,118553.0,11,2001,784731.0,435.36
46180,43851,562,154017.0,70.05,176796.0,11,2001,1142666.0,578.43
46109,172677,527,179032.0,63.05,193954.0,4,2002,1199661.0,450.55
32885,58955,642,148806.0,36.54,160597.0,5,2002,1086206.0,244.97
29841,38354,523,176634.0,62.01,215915.0,8,2002,1479162.0,494.77
38537,127183,643,152768.0,40.62,164904.0,10,2002,1086422.0,318.07
